# Fish in Image Detection

dataset: https://www.kaggle.com/datasets/slavkoprytula/aquarium-data-cots

Author: 'Hal' Sterling Halberstadt
Purpose: 
I want to make an object detection and this seemed like a difficult enough problem with a good enough dataset.

### Imports

In [1]:
import numpy as np
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers, Input, Model, callbacks, utils, callbacks, optimizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

from pathlib import Path
from PIL import Image # for resizing images
from IPython.display import display, HTML
from skimage import io, color

## Image set resizing
since the images in the dataset are different sizes, I am going to resize all the images beforehand so I don't need to add time resizing images in the generator 

The main thing is not to remove data, so that the text files are still accurate

NOTE: the code used to resize the images is in a markdown cell as for you to be able to replicate even though it will not run if directly pulled as is.

In [2]:
# Locations
data_dir = '../Comp_Vis_Proj/aquarium'

test_path = Path(data_dir+'/test/images').glob("*.jpg")
test_list = list(test_path)
train_path = Path(data_dir+'/train/images').glob("*.jpg")
train_list = list(train_path)
valid_path = Path(data_dir+'/valid/images').glob("*.jpg")
valid_list = list(valid_path)

# image constants
# img_size = (1024, 1024, 3)
# img_size = (512, 512, 3)
img_size = (256, 256, 3)

### Python code for resizing images:

note that this will pad the sides with black space to the right and bottom of the image

```
for current_list in [test_list, train_list, valid_list]:
    for i, ID in enumerate(current_list):
        # get image location
        image_location = str(ID)
        
        with Image.open(image_location) as image: # code adapted from https://jdhao.github.io/2017/11/06/resize-image-to-square-with-padding/
            old_size = image.size  # old_size[0] is in (width, height) format

            ratio = float(img_size[0])/max(old_size)
            new_size = tuple([int(x*ratio) for x in old_size])

            image = image.resize(new_size, Image.ANTIALIAS)

            new_im = Image.new("RGB", (img_size[0], img_size[0]))
            new_im.paste(image, ((img_size[0]-new_size[0])//2, (img_size[0]-new_size[1])//2))
            
            # new_im.show();
            new_im.save(image_location)
```

Now I need to grab the file(s) with the data, I am also then going to make a generator so that I do not need to hold more than a few images in memory at a time.

classes of fish associated with the dataset (pulled from the kaggle page listed above).

In [3]:
fish_classes = ['fish', 'jellyfish', 'penguin', 'puffin', 'shark', 'starfish', 'stingray']

### Useful functions

In [4]:
def plot_metric(history, metric='loss'): # credit to Glenn Bruns of CSUMB, this is taken from code provided during his ML course.
    """ Plot training and test values for a metric. """
    plt.figure(figsize=(4,4))
    val_metric = 'val_'+metric
    plt.plot(history.history[metric])
    plt.plot(history.history[val_metric])
    plt.title('model '+metric)
    plt.ylabel(metric)
    plt.xlabel('epoch')
    plt.legend(['train', 'test'])
    plt.show();

### Generators


In [5]:
class DataGenerator(utils.Sequence): 
    '''
    adapted from https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
    '''
    def __init__(self, list_IDs_fold, batch_size=8, 
                 dim=img_size, objs=(25, 5), n_channels=3, shuffle=True):
        self.dim = dim
        self.objs = objs
        self.batch_size = batch_size
        self.list_IDs = list_IDs_fold
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        # Find list of IDs
        list_IDs = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs)
        
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size, *self.objs))  
                
        
        for i, ID in enumerate(list_IDs_temp):
            # get image location
            image_location = str(ID)
            # print(image_location)
            # shift to fit correct location
            text_location = str(ID).replace("\\images\\", "\\labels\\").replace(".jpg", ".txt")
            
            num_coords = 0
            with open(text_location) as f:
                coord_txt = f.readline().rstrip()
                # print(f"\"{coord_txt}\"")
                if coord_txt == "":
                    y[i] = [-1.0, -1.0, -1.0, -1.0, -1.0]
                else:
                    y[i] = [float(i) for i in coord_txt.split()]
                # num_coords += 1
                
            # size halved in width and height to be possible to do.
            X[i] = load_img(image_location).resize(img_size[:2])
            # X[i] = load_img(image_location).resize((1024, 1024))
                    
        return X, y

In [6]:
 # Generators
test_generator = DataGenerator(test_list)
train_generator = DataGenerator(train_list)
valid_generator = DataGenerator(valid_list)

In [7]:
K.clear_session()
nodes = 8
_activation = 'relu'

# Training

In [8]:
inputs = Input((img_size), name="img_input")

x = layers.SeparableConv2D(nodes, (150,150), activation=_activation)(inputs)
x = layers.SeparableConv2D(nodes, (100,100), activation=_activation)(x)

x = layers.Reshape((8*8, nodes))(x)

x = layers.SeparableConv1D(8, (40), activation=_activation)(x)

x = layers.Dense(nodes, activation=_activation)(x)
x = layers.Dense(nodes*2, activation=_activation)(x)

x = layers.Dense(5, activation=_activation)(x)

model = Model(inputs, x)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img_input (InputLayer)      [(None, 256, 256, 3)]     0         
                                                                 
 separable_conv2d (Separable  (None, 107, 107, 8)      67532     
 Conv2D)                                                         
                                                                 
 separable_conv2d_1 (Separab  (None, 8, 8, 8)          80072     
 leConv2D)                                                       
                                                                 
 reshape (Reshape)           (None, 64, 8)             0         
                                                                 
 separable_conv1d (Separable  (None, 25, 8)            392       
 Conv1D)                                                         
                                                             

I want to create a loss function that is able to have the weights of correct fish Identification be able to be weighed differently than 

In [10]:
_monitor = 'val_loss'
_patience = 2
_verbose = 0
min_increase = 0.02
use_best = False
start_EarlyStopping = 3
scheduler_rate = -.1

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(scheduler_rate)

my_callbacks = [
    callbacks.EarlyStopping(monitor=_monitor,
                            min_delta=0,
                            patience=_patience,
                            verbose=_verbose,
                            baseline=min_increase,
                            restore_best_weights=use_best,
                            start_from_epoch=start_EarlyStopping),
    ReduceLROnPlateau(monitor=_monitor,
                      factor=0.1,
                      patience=_patience+1,
                      verbose=_verbose,
                      min_delta=0.0001,
                      cooldown=0,
                      min_lr=0),
    LearningRateScheduler(scheduler, 
                          verbose=_verbose)
]

In [ ]:
model.compile(optimizer='RMSprop', loss='mean_squared_error',  metrics=['accuracy'])

history = model.fit(train_generator, 
                    validation_data=valid_generator, # Note to self valid replaced test due to it being added later
                    callbacks=my_callbacks,
                    epochs=100, 
                    verbose=_verbose)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/100
56/56 [==============================] - 472s 8s/step - loss: 407210.2188 - accuracy: 0.5045 - val_loss: 2.1808 - val_accuracy: 0.6333 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/100
44/56 [======================>.......] - ETA: 1:41 - loss: 2.3174 - accuracy: 0.6534

In [ ]:
plot_metric(history)

Now that I have trained the model I want to try it on entirely new data and see how it does

In [ ]:
# model.evaluate(test_generator)
model.evaluate(valid_generator)

I am _ about how this turned out, I think I need to try doing _ and see how that might affect the accuracy.